# Analyzation of Gender Pay Gap by Flying Geckos
by Maryann Foley, Tiffany Moi, and Helen Ye

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
# import sklearn

In [44]:
data = pd.read_csv('data.csv')
for col in data.columns:
    if col == 'Occupation':
        continue
    data[col] = pd.to_numeric(data[col], errors='coerce')
data.head()

,Occupation,All_workers,All_weekly,M_workers,M_weekly,F_workers,F_weekly
0,ALL OCCUPATIONS,109080,809.0,60746,895.0,48334,726.0
1,MANAGEMENT,12480,1351.0,7332,1486.0,5147,1139.0
2,Chief executives,1046,2041.0,763,2251.0,283,1836.0
3,General and operations managers,823,1260.0,621,1347.0,202,1002.0
4,Legislators,8,NaN,5,NaN,4,NaN


In [31]:
data.shape

(558, 7)

In [45]:
# Extracts rows where the all the weekly wage data is present
all_wages = data[data.All_weekly.notnull() & data.M_weekly.notnull() & data.F_weekly.notnull()]
print(all_wages.shape)
all_wages.head()

(142, 7)


,Occupation,All_workers,All_weekly,M_workers,M_weekly,F_workers,F_weekly
0,ALL OCCUPATIONS,109080,809.0,60746,895.0,48334,726.0
1,MANAGEMENT,12480,1351.0,7332,1486.0,5147,1139.0
2,Chief executives,1046,2041.0,763,2251.0,283,1836.0
3,General and operations managers,823,1260.0,621,1347.0,202,1002.0
6,Marketing and sales managers,948,1462.0,570,1603.0,378,1258.0


In [40]:
sorted_worker = all_wages.sort_values(by=['F_workers', 'M_workers'])
sorted_worker.head()

,Occupation,All_workers,All_weekly,M_workers,M_weekly,F_workers,F_weekly
163,Editors,108,1148.0,58,1205.0,50,1125.0
451,"Food processing workers, all other",132,594.0,82,679.0,50,508.0
331,Postal service clerks,127,927.0,76,974.0,51,833.0
258,First-line supervisors of gaming workers,117,769.0,65,900.0,52,680.0
16,"Transportation, storage, and distribution mana...",276,966.0,224,1006.0,52,749.0


In [56]:
# I'm trying to figure out how to make another column with the percentage of female workers but I forg

-----
<built-in method count of tuple object at 0x7f6f35ad0ef0>
-----
<built-in method count of tuple object at 0x7f6f3590d878>
-----
<built-in method count of tuple object at 0x7f6f35ad0ef0>
-----
<built-in method count of tuple object at 0x7f6f3590d878>
-----
<built-in method count of tuple object at 0x7f6f35ad0ef0>
-----
<built-in method count of tuple object at 0x7f6f3590d878>
-----
<built-in method count of tuple object at 0x7f6f35ad0ef0>
-----
<built-in method count of tuple object at 0x7f6f3590d878>
-----
<built-in method count of tuple object at 0x7f6f35ad0ef0>
-----
<built-in method count of tuple object at 0x7f6f3590d878>
-----
<built-in method count of tuple object at 0x7f6f35ad0ef0>
-----
<built-in method count of tuple object at 0x7f6f3590d878>
-----
<built-in method count of tuple object at 0x7f6f35ad0ef0>
-----
<built-in method count of tuple object at 0x7f6f3590d878>
-----
<built-in method count of tuple object at 0x7f6f35ad0ef0>
-----
<built-in method count of tuple ob